In [17]:
# --------------------- Lucas-Kanade Optical Flow------------------------
import cv2
import numpy as np

# ---- INITIALIZATION ----
# Set up camera with lower resolution for better performance
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Parameters for Shi-Tomasi corner detection - reduced for better performance
feature_params = dict(maxCorners=50,      # Reduced number of corners to track
                    qualityLevel=0.3,     # Minimum quality level for corner detection
                    minDistance=10,       # Increased minimum distance between corners
                    blockSize=7)          # Size of window for corner detection

# Parameters for Lucas-Kanade optical flow - simplified
lk_params = dict(winSize=(15, 15),        # Size of search window
               maxLevel=2,                # Number of pyramid levels
               criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# ---- STEP 1: INITIALIZE FIRST FRAME ----
# Read initial frame
ret, old_frame = cap.read()
if not ret:
   print("Failed to grab first frame")
   exit()

# Reduce image size for faster processing
scale = 0.5
old_frame = cv2.resize(old_frame, None, fx=scale, fy=scale)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Detect initial points to track
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
if p0 is None:
   print("No points detected in first frame")
   exit()

# Initialize mask for drawing tracks
mask = np.zeros_like(old_frame)

while True:
   # STEP 2: CAPTURE NEW FRAME
   ret, frame = cap.read()
   if not ret:
       print("Failed to grab frame")
       break
   
   frame = cv2.resize(frame, None, fx=scale, fy=scale)
   frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   
   # STEP 3: CALCULATE OPTICAL FLOW
   p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
   
   if p1 is not None:
       good_new = p1[st == 1]
       good_old = p0[st == 1]
       
       mask = np.zeros_like(frame)
       
       # Draw motion tracks and current points
       for new, old in zip(good_new, good_old):
           a, b = new.ravel()
           c, d = old.ravel()
           mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), 
                         (0, 255, 0), 2)
           frame = cv2.circle(frame, (int(a), int(b)), 3, (0, 0, 255), -1)
       
       # Combine original frame with motion tracks
       lk_output = cv2.add(frame, mask)
       
       # Display sparse optical flow result
       cv2.imshow("Lucas-Kanade Optical Flow", lk_output)
       
       # Update for next frame
       old_gray = frame_gray.copy()
       p0 = good_new.reshape(-1, 1, 2)

   if cv2.waitKey(1) & 0xFF == ord('q'):
       break

# ---- CLEANUP ----
cap.release()
cv2.destroyAllWindows()

cv2.waitKey(1)

-1

In [13]:
#  ---------------------- Dense Optical Flow ----------------------
import cv2
import numpy as np

# Set up camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Read first frame
ret, old_frame = cap.read()
if not ret:
    print("Failed to grab first frame")
    exit()

# Scale down for better performance
scale = 0.5
old_frame = cv2.resize(old_frame, None, fx=scale, fy=scale)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

while True:
    # Read current frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Preprocess frame
    frame = cv2.resize(frame, None, fx=scale, fy=scale)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate Farneback dense optical flow
    flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None,
                                      0.5,  # Pyramid scale
                                      3,    # Pyramid levels
                                      15,   # Window size
                                      3,    # Iterations
                                      5,    # Poly_n
                                      1.2,  # Poly_sigma
                                      0)    # Flags
    
    # Visualize the flow
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 255  # Saturation
    
    # Calculate magnitude and angle of 2D vectors
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    
    # Convert to color
    hsv[..., 0] = ang * 180 / np.pi / 2  # Hue according to direction
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)  # Value according to magnitude
    
    # Convert HSV to BGR for display
    flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # Display result
    cv2.imshow('Dense Optical Flow', flow_rgb)
    
    # Update old frame
    old_gray = frame_gray.copy()
    
    # Break on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [16]:
# ---------------------- Lucas-Kanade Optical Flow and Dense Optical Flow with FPS ----------------------
import cv2
import numpy as np
import time

# ---- INITIALIZATION ----
# Set up camera with lower resolution for better performance
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Parameters for Shi-Tomasi corner detection - reduced for better performance
feature_params = dict(maxCorners=50,      # Reduced number of corners to track
                    qualityLevel=0.3,     # Minimum quality level for corner detection
                    minDistance=10,       # Increased minimum distance between corners
                    blockSize=7)          # Size of window for corner detection

# Parameters for Lucas-Kanade optical flow - simplified
lk_params = dict(winSize=(15, 15),        # Size of search window
               maxLevel=2,                # Number of pyramid levels
               criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# ---- STEP 1: INITIALIZE FIRST FRAME ----
# Read initial frame
ret, old_frame = cap.read()
if not ret:
   print("Failed to grab first frame")
   exit()

# Reduce image size for faster processing
scale = 0.5
old_frame = cv2.resize(old_frame, None, fx=scale, fy=scale)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Detect initial points to track
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
if p0 is None:
   print("No points detected in first frame")
   exit()

# Initialize mask for drawing tracks
mask = np.zeros_like(old_frame)

# Add mode flags and FPS variables
mode = 'original'  # 'original', 'lucas_kanade', or 'dense'
prev_frame_time = 0
new_frame_time = 0

while True:
    # Calculate FPS
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time) if prev_frame_time > 0 else 0
    prev_frame_time = new_frame_time
    fps_text = f"FPS: {int(fps)}"
    
    # Capture new frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    frame = cv2.resize(frame, None, fx=scale, fy=scale)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Check for key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('l'):
        mode = 'lucas_kanade'
    elif key == ord('d'):
        mode = 'dense'
    elif key == ord('o'):  # 'o' for original
        mode = 'original'
    elif key == ord('q'):
        break
    
    if mode == 'dense':
        # Dense Optical Flow
        flow = cv2.calcOpticalFlowFarneback(old_gray, frame_gray, None,
                                          0.5, 3, 15, 3, 5, 1.2, 0)
        
        hsv = np.zeros_like(frame)
        hsv[..., 1] = 255
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        output = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
    elif mode == 'lucas_kanade':
        # Lucas-Kanade Optical Flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]
            
            mask = np.zeros_like(frame)
            
            for new, old in zip(good_new, good_old):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), 
                             (0, 255, 0), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 3, (0, 0, 255), -1)
            
            output = cv2.add(frame, mask)
            p0 = good_new.reshape(-1, 1, 2)
        else:
            output = frame
            # Reset points if lost
            p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
            
    else:  # original mode
        output = frame
    
    # Add FPS and mode text to output
    cv2.putText(output, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
               1, (0, 255, 0), 2)
    cv2.putText(output, f"Mode: {mode}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 
               1, (0, 255, 0), 2)
    
    # Display output
    cv2.imshow("Camera View", output)
    
    # Update previous frame
    old_gray = frame_gray.copy()

# ---- CLEANUP ----
cap.release()
cv2.destroyAllWindows()

cv2.waitKey(1)

-1